# TensorFlow实现单层感知机详解

简单感知机是一个单层神经网络。它使用阈值激活函数，正如 Marvin Minsky 在论文中所证明的，它只能解决线性可分的问题。虽然这限制了单层感知机只能应用于线性可分问题，但它具有学习能力已经很好了。

当感知机使用阈值激活函数时，不能使用 TensorFlow 优化器来更新权重。我们将不得不使用权重更新规则：
$$
\Delta W = \eta X ^ { \mathrm { T } } \left( Y - Y _ { \mathrm { hat } } \right)
$$
η 是学习率。为了简化编程，当输入固定为 +1 时，偏置可以作为一个额外的权重。那么，上面的公式可以用来同时更新权重和偏置。

下面讨论如何实现单层感知机：

1. 导入所需的模块：

In [ ]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()


2. 定义要是用的超参数:

In [ ]:
# Hyper parameters
eta = 0.4 # learning rate parameter
epsilon = 1e-03 # minimum accepted error
max_epohs = 100 # Maximum Epochs

3. 指定训练数据。在这个例子中，取三个输入神经元（A，B，C）并训练它学习逻辑 AB+BC：

In [ ]:
# Training Data Y = AB + BC, sum of two linear functions.
T, F = 1.0, 0.0
X_in = [[T, T, T, T], [T, T, F, T], [T, F, T, T], [T, F, F, T], [F, T, T, T],
        [F, T, F, T], [F, F, T, T], [F, F, F, T]]

Y = [[T], [T], [F], [F], [T], [F], [F], [F]]

print(X_in)
print(Y)

4. 定义要用到的变量和用于计算更新的计算图，最后执行计算图：

In [ ]:
# Threshold Activation function
def threshold(x):
    """小于0返回0，否则返回1"""
    cond = tf.less(x, tf.zeros(tf.shape(x), dtype=x.dtype)) # cond： x 元素小于 0 的地方 为 True 否则 为 False
    out = tf.where(cond, tf.zeros(tf.shape(x)), tf.ones(tf.shape(x))) # out: cond 为True的地方是 0 是False的地方 是 1 
    return out

In [ ]:
w = tf.Variable(tf.random_normal([4, 1], stddev=2, seed=0))
h = tf.matmul(X_in, w)
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
Y_hat = threshold(h)
error = Y - Y_hat
mean_error = tf.reduce_mean(tf.square(error))
dW = eta * tf.matmul(X_in, error, transpose_a=True)
train = tf.assign(w, w + dW)


err = 1
epoch = 0
while err > epsilon and epoch < max_epohs:
    epoch += 1
    err, _ = sess.run([mean_error, train])
    print('epoch:{0} mean error:{1}'.format(epoch, err))
print('Training complete')

那么，如果使用 Sigmoid 激活函数，而不是阈值激活函数，会发生什么？你猜对了，首先，可以使用 TensorFlow 优化器来更新权重。其次，网络将表现得像逻辑回归。